In [ ]:
import pandas as pd
import datetime
pd.set_option("display.max_columns", None)



def generate_timestamp(format = '%y-%m-%dT%H-%M-%S'):
	ts = datetime.datetime.utcnow().strftime(format) # so you don't overwrite, and save timestamp
	return ts

In [ ]:


input_dir = './../../../data/ctl/'
output_dir = './data/input/ctl/'




set_names = ['train10_train_30perc' ,'train10_val_15perc','train10_test_15perc']

dataset_dir = '/Users/danielmlow/data/ctl/input/datasets/'

sub_dir = 'train10_subset_30'

# Text 
train = pd.read_parquet(dataset_dir + f'{sub_dir}/{set_names[0]}_messages_texter.gzip', engine='pyarrow')
val = pd.read_parquet(dataset_dir + f'{sub_dir}/{set_names[1]}_messages_texter.gzip', engine='pyarrow')
test = pd.read_parquet(dataset_dir + f'{sub_dir}/{set_names[2]}_messages_texter.gzip', engine='pyarrow')

# Metadata (i.e., target variables)
train_metadata = pd.read_csv(dataset_dir + f'{sub_dir}/{set_names[0]}_metadata.csv', index_col = 0)
val_metadata = pd.read_csv(dataset_dir+ f'{sub_dir}/{set_names[1]}_metadata.csv', index_col = 0)
test_metadata = pd.read_csv(dataset_dir + f'{sub_dir}/{set_names[2]}_metadata.csv', index_col = 0)




In [ ]:
test

In [ ]:
train_metadata

In [ ]:
# Concat messages

# merge with metadata
from tqdm import tqdm
dfs = {}


for split, messages_dv, metadata_df_i in zip(['train', 'val', 'test'], [train, val, test], [train_metadata, val_metadata, test_metadata]):
	messages_concat = []

	for i in  tqdm(metadata_df_i['conversation_id'].unique()):
		messages_i = []
		messages_dv_i = messages_dv[messages_dv['conversation_id']==i]
		if len(messages_dv_i) == 0:
			continue
		messages_dv_i = messages_dv_i.sort_values('message_timestamp_utc')
		messages_convo_i = [n.strip(' ') if n.endswith(('.', ',', ']', ')', '!','?', '>')) else n.strip(' ')+'.' for n in messages_dv_i['message'].tolist() ]
		X_i = ' '.join(messages_convo_i) # messages of 1 convo
		messages_i.append(i)
		messages_i.append(messages_dv_i['actor_id'].values[-1])
		messages_i.append(str(messages_dv_i['message_timestamp_utc'].values[-1]).replace('.000000', ''))
		messages_i.append(X_i)
		messages_concat.append(messages_i)

	messages_concat_df = pd.DataFrame(messages_concat, columns = ['conversation_id', 'actor_id', 'conversation_end_time_utc', 'message'])
	print('concat:',messages_concat_df.shape)
	metadata_df_i = metadata_df_i.merge(messages_concat_df, on='conversation_id', how='left', suffixes=('', ''))
	# drop if nan on message
	print(metadata_df_i.shape) 
	metadata_df_i = metadata_df_i[metadata_df_i['message'].notna()]
	print(metadata_df_i.shape) 
	
	dfs[split] = metadata_df_i.copy()
		

		





		

In [ ]:
for i, split in enumerate(['train', 'val', 'test']):
	metadata_df_i = dfs[split]
	metadata_df_i.to_csv(dataset_dir + f'{sub_dir}/{set_names[i]}_messages_texter_metadata.csv', index=False)

In [ ]:
metadata_df_i